In [5]:
!pip install pypdf

In [7]:
!pip install langchain_community

In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:\\Users\\chara\\Downloads\\Prompt.pdf")
docs = loader.load()

ValueError: File path C:\Users\chara\Downloads\Prompt.pdf is not a valid file or url

In [9]:
docs[3]

IndexError: list index out of range

In [11]:
docs[1]

IndexError: list index out of range

In [13]:
docs[0]

Document(metadata={'producer': 'macOS Version 15.2 (Build 24C101) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250211014202Z00'00'", 'moddate': "D:20250211014202Z00'00'", 'source': 'C:\\Users\\chara\\Downloads\\Prompt.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='In a world driven by data and innovation, I thrive at the intersection of cutting-edge technology and real-world problem-solving. As a Data Science professional with expertise in Gen AI, NLP, and scalable ML systems, I bring a results-driven mindset to every challenge. My recent tenure at Elevance Health saw the creation of a Gen AI-powered search engine, projected to deliver a $780M profit—a testament to my ability to merge strategy with execution.  My mobile number. Is 9999999999 and SSN Number is 123456789  I set all my passwords to Passwords@testing123 and my health insurance number is N123H456  Previously, as an Associate Software Engineer at Carelon Global Solutions, I led initiativ

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [71]:
all_splits

[Document(metadata={'producer': 'macOS Version 15.2 (Build 24C101) Quartz PDFContext', 'creator': 'PyPDF', 'creationdate': "D:20250211014202Z00'00'", 'moddate': "D:20250211014202Z00'00'", 'source': 'C:\\Users\\chara\\Downloads\\Prompt.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='In a world driven by data and innovation, I thrive at the intersection of cutting-edge technology and real-world problem-solving. As a Data Science professional with expertise in Gen AI, NLP, and scalable ML systems, I bring a results-driven mindset to every challenge. My recent tenure at Elevance Health saw the creation of a Gen AI-powered search engine, projected to deliver a $780M profit—a testament to my ability to merge strategy with execution.  My mobile number. Is 9999999999 and SSN Number is 123456789  I set all my passwords to Passwords@testing123 and my health insurance number is N123H456  Previously, as an Associate Software Engineer at Carelon Global Solutions, I led initiati

In [73]:
!pip install sentence-transformers

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\chara\AppData\Local\Temp\ipykernel_26984\4286477000.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [23]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [25]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model="llama3.2")

In [27]:
llm

OllamaLLM(model='llama3.2')

In [31]:
!pip install -qU langchain-community faiss-cpu

In [103]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# Create FAISS vector store
vectorstore = FAISS.from_documents(
    documents=all_splits,
    embedding=embeddings
)

In [125]:
vectorstore.save_local("faiss_index")

In [1]:
vectorstore = FAISS.load_local("faiss_index", embeddings)

NameError: name 'FAISS' is not defined

In [129]:
vector_store

In [107]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

C:\Users\chara\anaconda3\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [109]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [111]:
retriever = vector_store.as_retriever()

In [113]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000268CE3952B0>, search_kwargs={})

In [115]:
PROMPT_TEMPLATE = """You are an intelligent AI assistant with access to a knowledge base. Use the following retrieved context to answer the user’s question as accurately as possible.

Context: {context}

Question: {question}

Provide a clear and concise answer using the given context. If the context does not contain relevant information, simply say, "I don't have enough information to answer that."
"""

In [117]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=PROMPT_TEMPLATE
)

In [119]:
from langchain.chains import LLMChain, RetrievalQA
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

In [121]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [123]:
rag_chain.invoke("Give me my email?")

AttributeError: 'list' object has no attribute 'search'

In [79]:
rag_chain.invoke("hello world!")

"Hello! Unfortunately, I don't have enough information to provide an accurate answer based on the provided context. The context is a simple greeting, but it doesn't contain any specific question or topic to respond to. Can you please provide more context or clarify what you would like to know? I'll do my best to help!"